<a href="https://colab.research.google.com/github/asyraffatha/Task-MachineLearning/blob/main/Week%2014/Markov_dan_Hidden_Markov_Model_Asyraff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install hmmlearn
!pip install hmmlearn

# Import necessary libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from hmmlearn import hmm  # For Hidden Markov Model
from collections import Counter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 3.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = "/content/drive/MyDrive/Week 14/data.csv"  # Adjust the path
data = pd.read_csv(file_path, delimiter=';')


In [5]:
# Clean column names and process data
data.columns = data.columns.str.strip()
data = data.select_dtypes(include=[np.number]).dropna()
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [6]:
# Split features and labels
X = data_scaled[:, :-1]
y = data_scaled[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
# Markov Model
def train_markov(data):
    transition_counts = Counter((a, b) for a, b in zip(data[:-1], data[1:]))
    total_counts = Counter(data[:-1])
    transition_probs = {k: v / total_counts[k[0]] for k, v in transition_counts.items()}
    return transition_probs

In [9]:
# Hidden Markov Model (HMM)
def train_hmm(X, n_components=2):
    model = hmm.GaussianHMM(n_components=n_components, covariance_type="diag", n_iter=100)
    model.fit(X)
    return model

In [10]:
# RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, pooling="max"):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.pooling = pooling
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        if self.pooling == "max":
            x = torch.max(rnn_out, dim=1)[0]
        else:
            x = torch.mean(rnn_out, dim=1)
        return self.fc(x)

In [11]:
# Experiment Parameters
hidden_sizes = [16, 32, 64]
pooling_types = ["max", "avg"]
epochs_list = [5, 50, 100, 250, 350]
optimizers = ["SGD", "RMSProp", "Adam"]


In [12]:
# Training RNN
def train_rnn(hidden_size, pooling, optimizer_name, epochs):
    model = RNNModel(input_size=X_train.shape[1], hidden_size=hidden_size, output_size=1, pooling=pooling)
    optimizer = {"SGD": optim.SGD, "RMSProp": optim.RMSprop, "Adam": optim.Adam}[optimizer_name](model.parameters(), lr=0.01)
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X.unsqueeze(1))
            loss = criterion(outputs.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
    return model


In [13]:
# Main Script
results = []

In [16]:
# RNN Experiments
for hidden_size in hidden_sizes:
    for pooling in pooling_types:
        for optimizer_name in optimizers:
            for epochs in epochs_list:
                model = train_rnn(hidden_size, pooling, optimizer_name, epochs)
                model.eval()
                with torch.no_grad():
                    predictions = []
                    for batch_X, _ in test_loader:
                        pred = model(batch_X.unsqueeze(1))
                        predictions.extend(pred.squeeze().tolist())
                results.append({
                    "hidden_size": hidden_size,
                    "pooling": pooling,
                    "optimizer": optimizer_name,
                    "epochs": epochs,
                    "mse_loss": nn.MSELoss()(torch.tensor(predictions), y_test_tensor).item()
                })

In [17]:
# Save RNN Results
results_df = pd.DataFrame(results)
results_df.to_csv("rnn_results.csv", index=False)

In [18]:
# Markov Model Example
transition_probs = train_markov(y_train)
print("Markov Model Transition Probabilities:", transition_probs)

Markov Model Transition Probabilities: {(0.8891263042772198, 0.8891263042772198): 0.09876543209876543, (0.8891263042772198, 0.7657608350016585): 0.12345679012345678, (0.7657608350016585, -1.7896667428492534): 0.08941176470588236, (-1.7896667428492534, 0.14012166938988344): 0.15337423312883436, (0.14012166938988344, 0.347199421388147): 0.10129310344827586, (0.347199421388147, -0.40621112311617363): 0.10658307210031348, (-0.40621112311617363, 0.7657608350016585): 0.11347517730496454, (0.7657608350016585, -1.3755112388527264): 0.14588235294117646, (-1.3755112388527264, -1.3755112388527264): 0.10071942446043165, (-1.3755112388527264, 0.14012166938988344): 0.14148681055155876, (0.14012166938988344, 0.7657608350016585): 0.11853448275862069, (0.7657608350016585, 0.7657608350016585): 0.1388235294117647, (-1.3755112388527264, 1.545606837207885): 0.09832134292565947, (1.545606837207885, 1.545606837207885): 0.09060402684563758, (1.545606837207885, -1.7896667428492534): 0.08389261744966443, (-1.78

In [19]:
# Hidden Markov Model Example
hmm_model = train_hmm(X_train)
print("HMM Trained with n_components=2")

print("Experiments completed. RNN results saved to 'rnn_results.csv'.")

HMM Trained with n_components=2
Experiments completed. RNN results saved to 'rnn_results.csv'.
